In [1]:
import numpy as np
from sklearn.compose import ColumnTransformer
from scipy.special import boxcox1p
from scipy.stats import probplot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from category_encoders import *
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler

#### 1. Load data

In [21]:
folderPath = "../data/"
train = pd.read_csv(folderPath + "train.csv")  
test = pd.read_csv(folderPath + "test.csv")  
print(train.shape)
train.head(2)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


#### 2. EDA
- Check missing data
- Create extra features (day of week)
- Check feature distributions
- Check target variable distribution
- Check correlation(s)

##### Missing data

In [22]:
def checkMissingData(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data.head(10)

In [23]:
checkMissingData(train)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


- remove features with missing data > 40% & impute mean value into LotFrontage

In [24]:
train.drop(['PoolQC', 'MiscFeature', 'Alley', 'FireplaceQu', 'Fence'], axis=1, inplace = True)

- remove features with missing data > 40% & impute mean value into LotFrontage based on its own neighbourhood

In [48]:
df_ = train.groupby(["Neighborhood"]).LotFrontage.mean()
train[train.LotFrontage.isnull() == True].Neighborhood

Neighborhood
Blmngtn    47.142857
Blueste    24.000000
BrDale     21.562500
BrkSide    57.509804
ClearCr    83.461538
CollgCr    71.682540
Crawfor    71.804878
Edwards    68.217391
Gilbert    79.877551
IDOTRR     62.500000
MeadowV    27.800000
Mitchel    70.083333
NAmes      76.462366
NPkVill    32.285714
NWAmes     81.288889
NoRidge    91.878788
NridgHt    81.881579
OldTown    62.788991
SWISU      58.913043
Sawyer     74.437500
SawyerW    71.500000
Somerst    64.666667
StoneBr    62.700000
Timber     80.133333
Veenker    59.714286
Name: LotFrontage, dtype: float64

In [49]:
train["LotFrontage"] = train.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.mean()))

In [52]:
checkMissingData(train)

,Total,Percent
GarageYrBlt,81,0.055479
BsmtFinType2,38,0.026027
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtCond,37,0.025342
BsmtQual,37,0.025342
MasVnrArea,8,0.005479
MasVnrType,8,0.005479
Electrical,1,0.000685
SalePrice,0,0.000000


In [ ]:

train['LotFrontage'].fillna(train['LotFrontage'].mean(), inplace=True)

- 81 'Garage' variables are missing probably reflecting those houses without a garage
        - fill with 'NotAvail'

In [51]:
garage_cols = ['GarageType','GarageFinish', 'GarageCars','GarageQual', 'GarageCond']
train[garage_cols] = train[garage_cols].fillna('NotAvail')

- Recheck updated missing data

In [19]:
checkMissingData(train)

,Total,Percent
GarageYrBlt,81,0.055479
BsmtFinType2,38,0.026027
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtCond,37,0.025342
BsmtQual,37,0.025342
MasVnrArea,8,0.005479
MasVnrType,8,0.005479
Electrical,1,0.000685
SalePrice,0,0.000000


- Scale GarageYrBlt to values between 0 and 1 and set missing values to '0' 
        - thereby having little influence as a feature as they are literally houses with no garage

In [53]:
X = train.values
ct = ColumnTransformer([("mm", MinMaxScaler(), ['GarageYrBlt'])])
train.GarageYrBlt = ct.fit_transform(train)
train.GarageYrBlt = train.GarageYrBlt.fillna(0)
checkMissingData(train)

,Total,Percent
BsmtFinType2,38,0.026027
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtCond,37,0.025342
BsmtQual,37,0.025342
MasVnrType,8,0.005479
MasVnrArea,8,0.005479
Electrical,1,0.000685
SalePrice,0,0.000000
RoofStyle,0,0.000000


- The missing basement & MasVnr variables are for houses presumably without basements & masonry veneers: 
        - thereby having little influence as a feature as they are literally houses with no garage
        - use same method as before: type in "NotAvail"
        - replace missing electrical value with most common type

In [75]:
cols = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtCond', 'BsmtCond', 'BsmtQual', 'MasVnrType']
train.MasVnrArea.fillna(value='NotAvail', inplace=True)
for col in cols:
    train[col].fillna(value='NotAvail', inplace=True)
train.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [88]:
#train[train['BsmtExposure']].fillna(value='NotAvail', inplace=True)
train.Electrical.fillna(value='SBrkr', inplace=True)
checkMissingData(train).head(1)

,Total,Percent
SalePrice,0,0.0


No more missing data

In [ ]:
X = df.drop(['count', 'log_count'], axis = 1).values
    scaler = StandardScaler().fit_transform(X)
    df_scaled = pd.DataFrame(X, columns=df.columns[:-2])
    df_scaled['count'] = df['count']
    df_scaled['log_count'] = df.log_count

In [ ]:
from sklearn.compose import ColumnTransformer
>>> from sklearn.preprocessing import Normalizer
>>> ct = ColumnTransformer(
...     [("norm1", Normalizer(norm='l1'), [0, 1]),
...      ("norm2", Normalizer(norm='l1'), slice(2, 4))])
>>> X = np.array([[0., 1., 2., 2.],
...               [1., 1., 0., 1.]])
>>> # Normalizer scales each row of X to unit norm. A separate scaling
>>> # is applied for the two first and two last elements of each
>>> # row independently.
>>> ct.fit_transform(X)
array([[0. , 1. , 0.5, 0.5],
       [0.5, 0.5, 0. , 1. ]])